In [1]:
# Add the parent directory to sys.path
import sys, os
from pathlib import Path

current_directory = os.getcwd()
parent_directory = Path(current_directory).parent.parent
sys.path.insert(0, str(parent_directory))
sys.path

['c:\\Users\\mekbi\\Desktop\\Kifiya\\week-2\\telecom-data-analysis',
 'C:\\Users\\mekbi\\AppData\\Local\\Programs\\Python\\Python312\\python312.zip',
 'C:\\Users\\mekbi\\AppData\\Local\\Programs\\Python\\Python312\\DLLs',
 'C:\\Users\\mekbi\\AppData\\Local\\Programs\\Python\\Python312\\Lib',
 'C:\\Users\\mekbi\\AppData\\Local\\Programs\\Python\\Python312',
 'c:\\Users\\mekbi\\Desktop\\Kifiya\\week-2\\telecom-data-analysis\\venv',
 '',
 'c:\\Users\\mekbi\\Desktop\\Kifiya\\week-2\\telecom-data-analysis\\venv\\Lib\\site-packages',
 'c:\\Users\\mekbi\\Desktop\\Kifiya\\week-2\\telecom-data-analysis\\venv\\Lib\\site-packages\\win32',
 'c:\\Users\\mekbi\\Desktop\\Kifiya\\week-2\\telecom-data-analysis\\venv\\Lib\\site-packages\\win32\\lib',
 'c:\\Users\\mekbi\\Desktop\\Kifiya\\week-2\\telecom-data-analysis\\venv\\Lib\\site-packages\\Pythonwin']

### Load engagement and experience data

In [2]:
# Load engagement data
import pandas as pd

engagement_picke_path = f'{str(parent_directory)}/notebooks/user_engagement_analysis/engagement_data.pkl'
engagement_data = pd.read_pickle(engagement_picke_path)
engagement_data.head()

,MSISDN/Number,Session Dur. (ms),Session Frequency,Total Data (Bytes)
0,3.360100e+10,116720.0,1,8.786906e+08
1,3.360100e+10,181230.0,1,1.568596e+08
2,3.360100e+10,134969.0,1,5.959665e+08
3,3.360101e+10,49878.0,1,4.223207e+08
4,3.360101e+10,37104.0,2,1.457411e+09


In [3]:
# Load experience data
experience_pickle_path = f'{str(parent_directory)}/notebooks/user_experience_analysis/experience_data.pkl'
experience_data = pd.read_pickle(experience_pickle_path)
experience_data.head()

,MSISDN/Number,Avg RTT DL (ms),Avg RTT UL (ms),Avg Bearer TP DL (kbps),Avg Bearer TP UL (kbps),TCP DL Retrans. Vol (Bytes),TCP UL Retrans. Vol (Bytes),Handset Type
0,3.360100e+10,46.0,0.0,37.0,39.0,0.0,0.0,Huawei P20 Lite Huawei Nova 3E
1,3.360100e+10,30.0,1.0,48.0,51.0,0.0,0.0,Apple iPhone 7 (A1778)
2,3.360100e+10,0.0,0.0,48.0,49.0,0.0,0.0,undefined
3,3.360101e+10,69.0,15.0,204.0,44.0,1066.0,0.0,Apple iPhone 5S (A1457)
4,3.360101e+10,114.0,5.0,40395.0,103.0,9349630.0,21202.0,Apple iPhone Se (A1723)


In [4]:
experience_data = experience_data.drop('Handset Type', axis=1)
experience_data.columns

Index(['MSISDN/Number', 'Avg RTT DL (ms)', 'Avg RTT UL (ms)',
       'Avg Bearer TP DL (kbps)', 'Avg Bearer TP UL (kbps)',
       'TCP DL Retrans. Vol (Bytes)', 'TCP UL Retrans. Vol (Bytes)'],
      dtype='object')

### Assign scores to each dataset

##### Assign scores to engagement data using Euclidean Distance

In [5]:
# Get the centroids
from scripts import satisfaction_utils

engagement_centroids = satisfaction_utils.calculate_centroids(df=engagement_data.drop('MSISDN/Number', axis=1))
engagement_centroids

INFO:scripts.satisfaction_utils:Data normalization successful
INFO:scripts.satisfaction_utils:KMeans clustering successful
INFO:scripts.satisfaction_utils:Centroids calculated for 3 clusters


array([[ 2.73387952,  3.71325469,  3.24708014],
       [-0.23130937, -0.45376432, -0.39897854],
       [ 0.36878701,  0.97567411,  0.86060373]])

In [6]:
# Get the least engaged clusters centroid
least_engaged_centroid = engagement_centroids[1]
least_engaged_centroid

array([-0.23130937, -0.45376432, -0.39897854])

In [7]:
# Normalize the engagement data
from scripts import enagagement_utils

normalized_engagement_data = enagagement_utils.normalize_data(df=engagement_data.drop('MSISDN/Number', axis=1))
normalized_engagement_data

array([[0.00590825, 0.05555556, 0.09593143],
       [0.00938651, 0.05555556, 0.01402598],
       [0.0068922 , 0.05555556, 0.06385101],
       ...,
       [0.04692194, 0.05555556, 0.02256615],
       [0.01325784, 0.05555556, 0.06388747],
       [0.04651534, 0.05555556, 0.0120169 ]])

In [8]:
# Calculate engagement score for each user
engagement_scores = []
for user_data in normalized_engagement_data:
    score = satisfaction_utils.euclidean_distance(data=user_data,
                                                  centroid=least_engaged_centroid)
    engagement_scores.append(score)

engagement_scores[:10]

[np.float64(0.7487421574482148),
 np.float64(0.6985083949476356),
 np.float64(0.7282567643519451),
 np.float64(0.7143828277907733),
 np.float64(0.829207218440659),
 np.float64(0.7714664256227064),
 np.float64(0.7720632502592477),
 np.float64(0.7087692185419597),
 np.float64(0.8019232208420178),
 np.float64(0.7380210571261697)]

In [9]:
# Add the scores to the engagement dataframe
engagement_data['Engagement Score'] = engagement_scores
engagement_data.head()

,MSISDN/Number,Session Dur. (ms),Session Frequency,Total Data (Bytes),Engagement Score
0,3.360100e+10,116720.0,1,8.786906e+08,0.748742
1,3.360100e+10,181230.0,1,1.568596e+08,0.698508
2,3.360100e+10,134969.0,1,5.959665e+08,0.728257
3,3.360101e+10,49878.0,1,4.223207e+08,0.714383
4,3.360101e+10,37104.0,2,1.457411e+09,0.829207


##### Assign scores to experience data using Euclidean Distance

In [10]:
# Get the centroids
experience_centroids = satisfaction_utils.calculate_centroids(df=experience_data.drop('MSISDN/Number', axis=1))
experience_centroids

INFO:scripts.satisfaction_utils:Data normalization successful
INFO:scripts.satisfaction_utils:KMeans clustering successful
INFO:scripts.satisfaction_utils:Centroids calculated for 3 clusters


array([[ 0.55240988,  0.48108072,  0.72727337,  0.74357972,  0.53753922,
         0.59110857],
       [-0.32026842, -0.30621465, -0.37453871, -0.35283207, -0.30681278,
        -0.3183051 ],
       [ 3.78480242,  4.07338297,  3.64158201,  2.8661137 ,  3.54528117,
         3.3552507 ]])

In [11]:
# Get the least engaged clusters centroid
worst_experience_centroid = experience_centroids[1]
worst_experience_centroid

array([-0.32026842, -0.30621465, -0.37453871, -0.35283207, -0.30681278,
       -0.3183051 ])

In [12]:
# Normalize the engagement data
normalized_experience_data = enagagement_utils.normalize_data(df=experience_data.drop('MSISDN/Number', axis=1))
normalized_experience_data

array([[4.69867211e-02, 0.00000000e+00, 1.41506548e-04, 2.41830471e-03,
        0.00000000e+00, 0.00000000e+00],
       [3.06435138e-02, 4.73933649e-03, 1.83576062e-04, 3.16239846e-03,
        0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 1.83576062e-04, 3.03838284e-03,
        0.00000000e+00, 0.00000000e+00],
       ...,
       [0.00000000e+00, 0.00000000e+00, 3.82450129e-06, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 4.20695141e-05, 1.36417189e-03,
        0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 7.64900257e-06, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00]])

In [13]:
# Calculate engagement score for each user
experience_scores = []
for user_data in normalized_experience_data:
    score = satisfaction_utils.euclidean_distance(data=user_data,
                                                  centroid=worst_experience_centroid)
    experience_scores.append(score)

experience_scores[:10]

[np.float64(0.8310740434431517),
 np.float64(0.8260948140631375),
 np.float64(0.8116970277710301),
 np.float64(0.8705176258290518),
 np.float64(1.0711577018302727),
 np.float64(0.8482672706505855),
 np.float64(0.9796244815369334),
 np.float64(0.8514857814068123),
 np.float64(0.813468419457046),
 np.float64(0.8425838091921715)]

In [14]:
# Add the scores to the engagement dataframe
experience_data['Experience Score'] = experience_scores
experience_data.head()

,MSISDN/Number,Avg RTT DL (ms),Avg RTT UL (ms),Avg Bearer TP DL (kbps),Avg Bearer TP UL (kbps),TCP DL Retrans. Vol (Bytes),TCP UL Retrans. Vol (Bytes),Experience Score
0,3.360100e+10,46.0,0.0,37.0,39.0,0.0,0.0,0.831074
1,3.360100e+10,30.0,1.0,48.0,51.0,0.0,0.0,0.826095
2,3.360100e+10,0.0,0.0,48.0,49.0,0.0,0.0,0.811697
3,3.360101e+10,69.0,15.0,204.0,44.0,1066.0,0.0,0.870518
4,3.360101e+10,114.0,5.0,40395.0,103.0,9349630.0,21202.0,1.071158


### Merge engagement and experience datasets

In [15]:
satisfaction_data = pd.merge(engagement_data[['MSISDN/Number', 'Engagement Score']],
                             experience_data[['MSISDN/Number', 'Experience Score']],
                             on='MSISDN/Number')
satisfaction_data.head()

,MSISDN/Number,Engagement Score,Experience Score
0,3.360100e+10,0.748742,0.831074
1,3.360100e+10,0.698508,0.826095
2,3.360100e+10,0.728257,0.811697
3,3.360101e+10,0.714383,0.870518
4,3.360101e+10,0.829207,1.071158


In [16]:
# Calculate the satisfaction score
# The average of engagement and experience scores
satisfaction_data['Satisfaction Score'] = satisfaction_data[['Engagement Score', 'Experience Score']].mean(axis=1)
satisfaction_data.head()


,MSISDN/Number,Engagement Score,Experience Score,Satisfaction Score
0,3.360100e+10,0.748742,0.831074,0.789908
1,3.360100e+10,0.698508,0.826095,0.762302
2,3.360100e+10,0.728257,0.811697,0.769977
3,3.360101e+10,0.714383,0.870518,0.792450
4,3.360101e+10,0.829207,1.071158,0.950182


In [17]:
# Save satisfaction data to load in the regression model
satisfaction_data.to_pickle('satisfaction_data.pkl')

### Top 10 satisfied customers

In [18]:
# Sort by satisfaction score and get the top 10
top_10_satisfied = satisfaction_data.sort_values(by='Satisfaction Score', ascending=False).head(10)
top_10_satisfied

,MSISDN/Number,Engagement Score,Experience Score,Satisfaction Score
76363,3.367588e+10,1.888282,2.476768,2.182525
37470,3.365982e+10,1.362202,2.816665,2.089434
1279,3.360452e+10,1.597926,2.521047,2.059487
37052,3.365973e+10,1.901504,2.147969,2.024736
6437,3.361489e+10,2.121897,1.878807,2.000352
13180,3.362578e+10,2.306460,1.608008,1.957234
39120,3.366021e+10,1.397155,2.340010,1.868583
666,3.360313e+10,1.581619,2.116028,1.848824
13526,3.362632e+10,2.073547,1.588213,1.830880
35436,3.365936e+10,1.458756,2.125255,1.792006


##### Export Satisfaction data to database

In [19]:
# Import utility function
from scripts import utils

In [20]:
# Create a connection
CREDENTIALS = utils.load_environment_variables(parent_directory=parent_directory)
connection = utils.create_alchemy_connection(credentials=CREDENTIALS)

INFO:scripts.utils:Trying to load environment variables from c:\Users\mekbi\Desktop\Kifiya\week-2\telecom-data-analysis/.env
INFO:scripts.utils:Credentials loaded successfully


In [21]:
# Export data
table_name  = 'satisfaction_data'
utils.export_df_to_db(df=satisfaction_data,
                      conn=connection,
                      table_name=table_name,
                      if_exists='replace')

INFO:root:DataFrame exported to table 'satisfaction_data' successfully.


True